# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\tag{2}
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otoczenia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\tag{3}
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

**Odpowiedź**

Dla pikseli o podobnej jasności (różnica 0, 1, 2), wartość funkcji γ będzie bliska 1. Dla pikseli o znacznie różnej jasności (255, 200), wartość funkcji γ będzie znacznie mniejsza niż 1. Dokładna wartość funkcji zależy od wartości parametru $\delta_r$, który wpływa na zakres zmian wartości funkcji γ.

In [ ]:
from skimage.metrics import mean_squared_error, structural_similarity
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
from scipy import signal
import cv2
import math
import os
import copy

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate


In [ ]:
#TODO Samodzielna
mat = loadmat('MR_data.mat')
Input_0 = mat['I_noisefree']
Input_1 = mat['I_noisy1']
Input_2 = mat['I_noisy2']
Input_3 = mat['I_noisy3']
Input_4 = mat['I_noisy4']

plt.imshow(Input_3, 'gray')
plt.axis('off')
plt.show()

**Odpowiedź**

Jest to tomografia głowy, która jest jedną z technik obrazownia medycznego.

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
#TODO Samodzielna

def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g

In [ ]:
#TODO Samodzielna
def coef_1(window1, filtr, variancy):
    A, B = window1.shape
    coefficient = 0
    x = [A//2,B//2]
    for i in range(A):
        for j in range(B):
            AB=[i, j]
            y=np.sqrt(((x[0]-AB[0])**2)+((x[1]-AB[1])**2))
            gaus= np.exp(-(y**2)/(2*(variancy**2)))
            coefficient += gaus * window1[i, j]
    coefficient /= filtr.sum()
    return coefficient

def conv(img, window2, variancy):
    filtr = fgaussian(5,variancy)
    IConv = img.copy()
    X,Y = IConv.shape
    half = window2//2
    for i in range(half, X-half):
        for j in range(half, Y-half):
            window1 = IConv[i-half:i+half+1,j-half:j+half+1]
            new_coefficient = coef_1(window1,filtr,variancy)
            IConv[i, j] = new_coefficient
    return IConv
        
Input_3_fil_conv = conv(Input_3, 5, 0.9)
f, (ax1,ax2) = plt.subplots(1,2,figsize=(18,8))
ax1.set_title("Obraz oryginalny")
ax1.imshow(Input_3, cmap='gray')
ax1.axis('off')
ax2.set_title("Obraz po filtracji")
ax2.imshow(Input_3_fil_conv, cmap='gray')
ax2.axis('off')
plt.show()

In [ ]:
# Obliczenie błędu średniokwadratowego (MSE)
MSE = mean_squared_error(Input_3, Input_3_fil_conv)

# Obliczenie indeksu podobieństwa strukturalnego (SSIM)
SSIM = structural_similarity(Input_3, Input_3_fil_conv)

# Wyświetlenie wyników
print("Błąd średniokwadratowy (MSE): ", MSE)
print("Indeks podobieństwa strukturalnego (SSIM): ", SSIM)

**Odpowiedź**

Na podstawie wyników błędu średniokwadratowego (MSE) i indeksu podobieństwa strukturalnego (SSIM) można stwierdzić, że konwolucja przyczyniła się do znacznej redukcji szumu, ale jednocześnie wprowadziła pewne zniekształcenia w obrazie, co obniżyło jego jakość w porównaniu do oryginału.

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gaussowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def coef_2(window3, filtr, variancy, delta):
    A, B = window3.shape
    normalization = 0
    coefficient = 0
    x = [A//2,B//2]
    for i in range(A):
        for j in range(B):
            AB = [i, j]
            y = np.sqrt(((x[0]-AB[0])**2) + ((x[1]-AB[1])**2))
            gaus = np.exp(-(y**2)/(2*(variancy**2)))
            diff = np.abs(window3[A//2, B//2] - window3[i, j])
            diff_gaus = np.exp(-(diff**2)/(2*(delta**2)))
            coefficient += gaus * diff_gaus * window3[i, j]
            normalization += gaus * diff_gaus
    coefficient /= normalization
    return coefficient

def bilateral(img, window4, variancy, delta):
    filtr = fgaussian(window4, variancy)
    IBilateral = img.copy()
    X,Y = IBilateral.shape
    half = window4//2
    for i in range(half, X-half):
        for j in range(half, Y-half):
            window3 = IBilateral[i-half:i+half+1,j-half:j+half+1]
            new_coefficient = coef_2(window3, filtr, variancy, delta)
            IBilateral[i,j] = new_coefficient
    return IBilateral

In [ ]:
Input_3_fil_bil = bilateral(Input_3, 5, 0.9, 25)
f, (ax1,ax2) = plt.subplots(1,2,figsize=(18,8))
ax1.set_title("Obraz oryginalny")
ax1.imshow(Input_3, cmap='gray')
ax1.axis('off')
ax2.set_title("Obraz po filtracji")
ax2.imshow(Input_3_fil_bil, cmap='gray')
ax2.axis('off')
plt.show()

In [ ]:
# Obliczenie błędu średniokwadratowego (MSE)
MSE = mean_squared_error(Input_3, Input_3_fil_bil)

# Obliczenie indeksu podobieństwa strukturalnego (SSIM)
SSIM = structural_similarity(Input_3, Input_3_fil_bil)

# Wyświetlenie wyników
print("Błąd średniokwadratowy (MSE): ", MSE)
print("Indeks podobieństwa strukturalnego (SSIM): ", SSIM)

**Odpowiedź**

Filtracja bilateralna usunęła znaczną ilość szumu z obrazu, zachowując jednocześnie dużą ilość szczegółów, co potwierdzają wartości MSE i SSIM.